In [1]:
%reload_ext autoreload
%autoreload

In [22]:
%autoreload now
from source.script import script_settings
options = script_settings("config.yml")

AppLayout(children=(VBox(children=(HTML(value='<h1>Script Settings</h1>'), HTML(value='<p>Relates to all the s…

In [23]:
%autoreload now
from source.script import setup_simulation

params = setup_simulation(options)

[2023-05-22 21:38:05,870 | INFO | script.__set_paths] Fetching directory paths
[2023-05-22 21:38:05,872 | INFO | script.__set_paths] Creating new path data
[2023-05-22 21:38:05,873 | INFO | script.__set_paths] Completed and returning paths
[2023-05-22 21:38:05,874 | INFO | script.__generate_dir_skeleton] Creating directory paths
[2023-05-22 21:38:05,876 | INFO | script.__generate_dir_skeleton] Making directories
[2023-05-22 21:38:05,884 | INFO | script.__generate_dir_skeleton] Path data saved to file
[2023-05-22 21:38:05,884 | INFO | script.__set_thread_count] Setting thread count to 24
[2023-05-22 21:38:05,886 | INFO | script.__set_random_seed] Setting random state seed to 666
[2023-05-22 21:38:05,887 | INFO | script.__set_matplotlib_dir] Matplotlib config directory unchanged
[2023-05-22 21:38:05,887 | INFO | script.__set_params] Creating new parameter data
[2023-05-22 21:38:05,906 | INFO | script.__set_params] Completed and returning parameters
[2023-05-22 21:38:05,907 | INFO | scrip

In [24]:
%autoreload now
from source.data import measurement_set_options

ms_options = measurement_set_options(params)

AppLayout(children=(VBox(children=(HTML(value='<h1>Measurement Set</h1>'), HTML(value='<p>Settings to generate…

In [25]:
%autoreload now
from source.data import create_empty_measurement_set

create_empty_measurement_set(ms_options, params)

[2023-05-22 21:38:18,670 | INFO | data.create_empty_measurement_set] Retrieving measurement set options
[2023-05-22 21:38:18,672 | INFO | data.create_empty_measurement_set] Name/Path: `/net/sinatra/vault2-tina/welman/meerkat-src-100/meerkat.ms`
[2023-05-22 21:38:18,673 | INFO | data.create_empty_measurement_set] Telescope: meerkat
[2023-05-22 21:38:18,674 | INFO | data.create_empty_measurement_set] Right Ascension: 11h50m15s
[2023-05-22 21:38:18,674 | INFO | data.create_empty_measurement_set] Declination: -30d27m43s
[2023-05-22 21:38:18,675 | INFO | data.create_empty_measurement_set] Synthesis-Time: 2 hr
[2023-05-22 21:38:18,676 | INFO | data.create_empty_measurement_set] Integration-Time: 10 s
[2023-05-22 21:38:18,677 | INFO | data.create_empty_measurement_set] Initial Frequency: 1GHz


[Warning] The following `1` files/folders already exist:
	`/net/sinatra/vault2-tina/welman/meerkat-src-100/meerkat.ms`


[2023-05-22 21:38:30,385 | INFO | data.create_empty_measurement_set] Create measurement set with `simms`
[2023-05-22 21:38:52,009 | INFO | data.create_empty_measurement_set] Updating path and parameter data
[2023-05-22 21:38:52,117 | INFO | data.create_empty_measurement_set] New measurement set created at `meerkat.ms`


In [26]:
%autoreload now
from source.data import gains_options

gs_options = gains_options(params)

AppLayout(children=(VBox(children=(HTML(value='<h1>Gains Signal</h1>'), HTML(value='<p>\n        These setting…

In [30]:
%autoreload now
from source.data import create_gains_signal

create_gains_signal(gs_options, params)

[2023-05-22 21:42:42,617 | INFO | data.create_gains_signal] Setting seed to 666
[2023-05-22 21:42:42,620 | WARNING | other.check_for_data] The following `1` files/folders already exist:
[2023-05-22 21:42:42,620 | WARNING | other.check_for_data] Delete? `/net/sinatra/vault2-tina/welman/meerkat-src-100/gains/true/true-gains.npy`
[2023-05-22 21:42:52,950 | INFO | data.create_gains_signal] Retrieving information from `/net/sinatra/vault2-tina/welman/meerkat-src-100/meerkat.ms`
[2023-05-22 21:42:53,030 | INFO | data.create_gains_signal] Calculating dimensions and axes
[2023-05-22 21:42:53,056 | INFO | data.create_gains_signal] Generating full-complex gains
[2023-05-22 21:42:53,181 | INFO | data.create_gains_signal] Codex shape: (N_ant, N_time, N_chan, N_dir, N_corr) = (720, 64, 1, 1, 1)
[2023-05-22 21:42:53,183 | INFO | data.create_gains_signal] Saving gains to file
[2023-05-22 21:42:53,192 | INFO | data.create_gains_signal] Updating path data
[2023-05-22 21:42:53,210 | INFO | data.create_g

In [40]:
%autoreload now
from source.data import load_gains

true_path = params["paths"]["gains"]["true"]
load_gains(true_path)

[2023-05-22 21:56:48,756 | INFO | data.load_gains] Loading gains from `/net/sinatra/vault2-tina/welman/meerkat-src-100/gains/true/true-gains.npy`


array([[1.12583001-0.29892048j, 1.09853714-0.43421508j,
        0.95771373-0.19630535j, ..., 1.04345548+0.25792208j,
        1.07712212-0.04855403j, 1.24536843+0.1963869j ],
       [1.12702758-0.29811656j, 1.09493215-0.43749483j,
        0.9629977 -0.16559095j, ..., 1.03349586+0.29383322j,
        1.07925066-0.05511208j, 1.25155617+0.16024067j],
       [1.12943472-0.292746j  , 1.09338528-0.4355681j ,
        0.96765988-0.13197966j, ..., 1.02344575+0.32580346j,
        1.08154621-0.05799211j, 1.25687151+0.12206649j],
       ...,
       [0.71380286+0.09692549j, 0.92539676+0.01021641j,
        0.73208083-0.07984167j, ..., 1.00778076-0.02281715j,
        1.26283634+0.15176847j, 0.93375365+0.17875298j],
       [0.71440742+0.07712583j, 0.92690013-0.0026851j ,
        0.73468941-0.05985389j, ..., 1.0069072 -0.05609225j,
        1.25845022+0.17197023j, 0.93760568+0.17375523j],
       [0.71463691+0.05560338j, 0.92825174-0.01388792j,
        0.73696724-0.03582264j, ..., 1.00526195-0.0859037j ,
 